# Device location detection using noisy pairwise data 

## Structure of the document: 

1. [Problem statement](#problem_statement)
2. [Analogy to mechanical system](#mechanical_system)
3. [Mathematical model](#mathematical_model)
4. [Data generation](#data_generation)
5. [Mathematical Model Code](#model_code)
6. [Main Loop for Simulation](#main_loop)

# Problem statement <a name="problem_statement"></a>

We have an $N$ device. Each devices pair can measure the pairwise 
distance between them and return a measured value.  Measurements 
can be noisy and contain outliers.  We are presented with measured 
pairwise distance data. The goal is to find the approximate 
coordinates of each device. 


We can call "true, state", "true distances" and "true coordinates" to actual distances and coordinates of devices, measured without noise. 


# Analogy to mechanical system  <a name="mechanical_system"></a>

The problem can be visually represented as a collection of dots in space.  We receive the measured distances between all pairs of points. We should note, that the reconstructed system state (distances) from pairwise noisy data should be close to the true state.  

Now consider each pair's distance. Reconstructed distance should be almost equal to true distance. In this case, we can conclude the following: 
* If the noisy distance is less than the true distance, the reconstructed distance should be greater than the noisy. This means pair points should move away from each other.  <br />
* In the opposite case, when the noisy distance is greater than the true distance, the reconstructed distance should be less than the noisy. This means pair points should move closer to each other.  <br />
* And the final goal is that the points should be positioned to minimize the mean difference between true and reconstructed distances.

This system is very similar to a mechanical system. We can draw an analogy. We place N points in the space and connect each pair through the springs:
* Place stings for each pair we know the noisy distance.
* The unstretched length of each spring should be equal to the noisy distance between the pair it connects. 
* Take the specific stiffness of the string as constant. In this case, the displacement (squeeze/stretch) of a long spring by constant value creates a smaller force than the same displacement for a short spring. 
Last step is advisable because the error of measuring long distances is always greater and it should contribute with less proportion than error on short distances. 


In the initial state, we can choose the random placement of points in the space and connect all pairs with springs.
It is not necessary to place each point close to the true coordinates. Although placing points near true coordinates reduce solving time. 

From any (non-zero) initial point placements, the system will advance into equilibrium, where the net force to each point is zero.  
If all distances in the matrix are accurate and there is no noise, then all springs in equilibrium are unstretched.  <br />
In the case of noisy data, the springs in the equilibrium are stretched, although as we mentioned the sum of the forces acting on each point is zero.

For solving the problem we should take following steps: 
* Step 1. Initialize points in the space
* Step 2. Calculate Forces acting to each point
* Step 3. Update point coordinates using forces calculated in step 2
* Step 4. Repeat steps 2 and 3 until the system goes to equilibrium or the maximum number of iterations is reached

By our definition equilibrium is achieved when the maximum net force on any point in the system becomes very small (less than $10^{-4}$). 

Problem can be stated and solved in 1-dimensional **(1D)** 2-dimensional **(2D)** and 3-dimensional **(3D)** spaces. I will start from **1D** and generalize the solution to higher dimensions. 

# Mathematical model <a name="mathematical_model"></a>

We can number devices from $1$ to $N$. In this case, input pairwise data can be represented as $NxN$ matrix. The matrix element in place $(i,j)$ is the distance between $i$-th and $j$-th devices.  <br />

>$D =
 \begin{pmatrix}
  d_{1,1} & d_{1,2} & \cdots & d_{1,n} \\
  d_{2,1} & d_{2,2} & \cdots & d_{2,n} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  d_{n,1} & d_{n,2} & \cdots & d_{n,n}
 \end{pmatrix}$  <br /> 
 
 The matrix is symmetrical $d_{i,j} = d_{j,i}$ <br />
 Diagonal elements ($d_{1,1} = d_{2,2} = \cdots = d_{n,n} = 0 $) are zero. 
 ![Distance Matrix](images/distance_matrix.png)
 **Image 1:**: How to construct distance matrix using 3 devices
 

Distance between $i$ and $j$ points is euclidean distance and in Cartesian coordinates is equal to:
>$l_{i,j} = |x_i - x_j|$ for 1D  <br />
>$l_{i,j} = \sqrt{(x_i - x_j)^2 + (y_i - x_j)^2}$ for 2D <br />
>$l_{i,j} = \sqrt{(x_i - x_j)^2 + (y_i - x_j)^2 + (z_i - z_j)^2}$ for 3D  <br />

Where $(x_i, y_i, z_i)$ is current coordinate for $i$-th point. 

The next step is to calculate the net force acting on each point. For this purpose, we need to calculate the force each point acts to another point. We construct Force Matrix  $F$. Element $F_{i,j}$ is the force acting to the $i$-th point from the $j$-th point.  <br />
As we use springs, force is proportional to the strech of each spring $\Delta_{i,j} = d_{i,j}-l_{i,j}$.  Here $d_{i,j}$ is distance from noisy distance matrix and $l_{i,j}$ is actual distance calculated above.  <br />
As specific stiffnes of the spring is constant, force can be written in the form: $F_{i,j} = \Delta_{i,j}/d_{i,j}$. We should note that force $F_{i,j}$ is a **Vector** and we need to find components in the Cartesian coordinates. 

>$F =
 \begin{pmatrix}
  F_{1,1} & F_{1,2} & \cdots & F_{1,n} \\
  F_{2,1} & F_{2,2} & \cdots & F_{2,n} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  F_{n,1} & F_{n,2} & \cdots & F_{n,n}
 \end{pmatrix}$  <br /> 
 The matrix is antisymmetric $F_{i,j} = -F_{j,i}$ <br />
 Diagonal elements ($F_{1,1} = F_{2,2} = \cdots = F_{n,n} = 0 $) are zero. 
 
 
![Points connected with springs](images/springs.png)
**Image 2:**: 4 points connected with springs. Springs connecting (1,4) and (3,4) are squeezed, all other springs are stretched. 
 
Force equation can be written as: <br />
For **1D**: 
>$F_{i,j}= (l_{i,j} - d_{i, j})/ d_{i, j} * sgn (x_j - x_i)$  <br />

For **2D**: 
>$F_{i,j,x} = (l_{i,j} - d_{i,j})/ d_{i, j} * \cos (\alpha) * sgn (x_j - x_i)$ for x coordinate  <br />  
>$F_{i,j,y} = (l_{i,j} - d_{i, j})/ d_{i, j} * \sin (\alpha) * sgn (x_j - x_i)$ for y coordinate <br />  

where $ \alpha= arctan((y_j - y_i)/(x_j - x_i))$ is angle. $l_{i,j}$ is defined above. $sgn$ is sign function. 

For **3D**: 
>$F_{i,j,x} = sgn (x_j - x_i)  * (l_{i,j} - d_{i,j})/ d_{i, j} * \cos (\beta) * \cos (\alpha) * sgn (x_j - x_i)$ for x coordinate  <br />  
>$F_{i,j,y} = sgn (x_j - x_i)  * (l_{i,j} - d_{i, j})/ d_{i, j} * \cos (\beta) * \sin (\alpha) * sgn (x_j - x_i)$ for y coordinate   <br />  
>$F_{i,j,z} = (l_{i,j} - d_{i, j})/ d_{i, j} * \sin (\beta)$ for y coordinate in 2D   <br />  for z coordinate   <br />  

where $ \alpha= arctan((y_j - y_i)/(x_j - x_i))$ and $ \beta= arctan((z_j - z_i)/\sqrt{(x_j - x_i)^2 +(y_j - y_i)^2})$

After calculating Force Matrix we use $tanh$ activation function to scale forces. It makes model robust against outliers and learning process becomes smooth. 

After scaling we calculate net force by summing all indexes: 
>$F_{i} = \sum_{j=1}^{n} F_{i,j}$

Finally, we can update the coordinates of $i$-th point by dragging the point toward the force direction. 
>$X_i = X_i + F_i*dt$

We can call $dt$ a learning rate. 

# Data generation <a name="data_generation"></a>

For algorithm testing, it is necessary to have data suitable for the problem statement. To test algorithms for different scenarios different datasets may be generated for every run. 

Datasets are generated using the following steps:
* Manually define the number of devices 
* Manually define dimension (**1D**, **2D**, or **3D**) 
* Generate random coordinates for each device 
* Build **Distance Matrix** from generated coordinates
* Add noise to **Distance Matrix**
* Sort **Distance Matrix**

Problem Definition states that only pairwise distance data is available, and we discard generated coordinates for algorithm testing.

Distances should be noisy on every scale. It is achievable by adding noise that consists of two parts. 
* The first part is **absolute error**, taken from the uniform distribution with range $(-10, 10)$. For small distances, absolute error dominates. 
* The second part is a **relative error**, uniformly distributed in range $(-20\%, 20\%)$. For large distances, relative error may become the main source of the noise. 

**Distance Matrix** columns and rows are sorted in a way, that 0 row and 0 column are in ascending order. Sorting is reindexing a symmetrical matrix. We find the pair with the highest distance, sort devices by distance, and reindex matrix (change device numbers).  

In [ ]:
%matplotlib nbagg
# use nbagg or auto for simulation animation
import numpy as np
import matplotlib.pyplot as plt


def get_coordinates(n=10, dims=1):
    """Returns n random coordinates with dimension 'dims' """
    arr = np.random.randint(0, 10*n + 200, size=(n, dims))
    arr -= np.min(arr, axis=0)  # Shift array, coordinates are greater than 0
    return arr


def calc_dist_matrix(coords):
    """Calculates distance matrix using coordinates with any dimension"""
    dist_mat = np.zeros(shape=(len(coords), len(coords)))
    for i, crd_i in enumerate(coords):
        for j, crd_j in enumerate(coords[:i]):
            dist_mat[i, j] = np.sqrt(sum([(x_i - x_j)**2 for x_j, x_i in zip(crd_i, crd_j)]))
            dist_mat[j, i] = dist_mat[i, j]
    return dist_mat


def add_noise(dist_mat, abs_err=10, rel_err=0.20):
    """Adds relative and absolute error to distance matrix"""
    noisy = dist_mat.copy()
    for i, x_i in enumerate(noisy):
        for j, x_j in enumerate(noisy[:i]):
            noisy[i, j] *= (1+np.random.uniform(low=-rel_err, high=rel_err))  # Add relative error. ±20% by default
            noisy[i, j] += np.random.uniform(low=-abs_err, high=abs_err)  # Add absolute error. Range (-10,10) by default
            noisy[j, i] = noisy[i, j]  # Update symmetrical value
    return noisy

def sort_dist_matrix(dist_mat):
    """Sorts distance matrix by reindexing. Returns sorted distance matrix and indexes"""
    max_idx = np.unravel_index(np.argmax(dist_mat, axis=None), dist_mat.shape)[0]  # Find index of farthest point
    sort_idxs = np.argsort(dist_mat[max_idx])  # Index order for sorting
    dist_sorted = dist_mat[sort_idxs].T[sort_idxs]  # Reindex distance matrix
    return dist_sorted, sort_idxs

## Data generation example: 

In [ ]:
n = 10
dims = 2

coordinates = get_coordinates(n, dims)  # Create n=10 device coordinates in 2D.
distances = calc_dist_matrix(coordinates)  # Calculate distances between points
sorted_distances, sorted_indexes = sort_dist_matrix(distances)  # Sort distance matrix
noisy_distances = add_noise(sorted_distances)  # Add noise

average_noise = np.average(np.abs(noisy_distances-sorted_distances))  # Calculate Mean Absolute Noise
average_distance = np.average(distances)
print('Average distance is {:0.2f}'.format(average_distance))
print('Mean absolute difference between original and noisy distances is {:0.2f}'.format(average_noise))

In [ ]:
%precision 1
print('Distance Matrix :\n', distances)
print('\nSorted Distance Matrix :\n', sorted_distances)
print('\nIndexes used for sorting Distance Matrix', sorted_indexes)

plt.plot(*zip(*coordinates), 'o')
plt.title('Device Coordinates in 2D')
plt.show()

# Mathematical Model Code <a name="model_code"></a>

In [ ]:
def generate_starting_points(dist_mat, n, dims):
    """Generates initial placement points. Points are evenly distributed to line (1D), to circle (2D) """
    if dims == 1:
        return np.array([[i] for i in np.linspace(0, np.max(dist_mat), n)])

    elif dims == 2:  # select coordinates on the circle
        r = np.max(dist_mat) / 4  # initialize in the middle of the
        center = np.mean(dist_mat)
        d_alpha = 2 * np.pi / len(dist_mat)
        init_coords = np.array([[center + r * np.cos(d_alpha * i), center + r * np.sin(d_alpha * i)] for i in range(len(dist_mat))])
        return init_coords
    
    elif dims == 3: # return random coordinates
        return np.random.randint(0, np.max(dist_mat), size = (len(dist_mat), 3) )


def force_1D(coords, dist_mat):
    """Calculates resultant force in 1D"""
    force_mat = np.zeros(shape=dist_mat.shape)
    for i, x_i in enumerate(coords):
        for j, x_j in enumerate(coords[:i]):
            if dist_mat[i, j] is not None:  # if we have distance info/ No missing value
                force_mat[i, j] = np.tanh(5 * np.sign(x_j - x_i) * (np.abs(x_j - x_i) / dist_mat[i, j] - 1))
                force_mat[j, i] = - force_mat[i, j]
            else:  # if we dont have info about distance force is 0
                force_mat[i, j] = 0
                force_mat[j, i] = 0
    force = np.sum(force_mat, axis=1)
    return np.array([[i] for i in force])


def force_2D(coords, dist_mat):
    """Calculates resultant force in 2D"""
    force_mat = np.zeros(shape=(len(coords), len(coords), 2))
    for i, (x_i, y_i) in enumerate(coords):
        for j, (x_j, y_j) in enumerate(coords[:i]):
            dx = x_j - x_i + 0.001
            dy = y_j - y_i + 0.001
            alpha = np.arctan(dy / dx)
            #             print(dx, dy, alpha)
            dl = np.sqrt(dx ** 2 + dy ** 2) - dist_mat[i, j]

            if dist_mat[i, j] is not None:  # if we have distance info/ No missing value
                force_mat[i, j, 0] = np.tanh(dl / dist_mat[i, j] * np.cos(alpha) * np.sign(dx))
                force_mat[i, j, 1] = np.tanh(dl / dist_mat[i, j] * np.sin(alpha) * np.sign(dx))
                force_mat[j, i, :] = - force_mat[i, j, :]
            else:  # if we dont have info about distance force is 0
                force_mat[i, j] = 0
                force_mat[j, i] = 0

    force = np.sum(force_mat, axis=1)
    return force

def force_3D(coords, dist_mat):
    """Calculates resultant force in 3D"""
    force_mat = np.zeros(shape=(len(coords), len(coords), 3))
    for i, (x_i, y_i, z_i) in enumerate(coords):
        for j, (x_j, y_j, z_j) in enumerate(coords[:i]):
            dx = x_j - x_i + 0.001
            dy = y_j - y_i + 0.001
            dz = z_j - z_i + 0.001
            alpha_1 = np.arctan(dz/np.sqrt(dx**2 + dy**2))
            alpha_2 = np.arctan(dy/dx)
            dl = np.sqrt(dx**2 + dy**2 + dz**2) - dist_mat[i, j]

            if dist_mat[i, j] is not None:  # if we have distance info/ No missing value
                force_mat[i, j, 0] = np.tanh(dl / dist_mat[i, j] * np.cos(alpha_1) * np.cos(alpha_2) * np.sign(dx))
                force_mat[i, j, 1] = np.tanh(dl / dist_mat[i, j] * np.cos(alpha_1) * np.sin(alpha_2) * np.sign(dx))
                force_mat[i, j, 2] = np.tanh(dl / dist_mat[i, j] * np.sin(alpha_1))
                force_mat[j, i, :] = - force_mat[i, j, :]
            else:  # if we dont have info about distance force is 0
                force_mat[i, j] = 0
                force_mat[j, i] = 0

    force = np.sum(force_mat, axis=1)
    return force

def calculate_force(coords, dist_mat):
    """Calculates resultant force acted on each point"""
    if len(coords[0]) == 1:  # 1D
        force = force_1D(coords, dist_mat)
    elif len(coords[0]) == 2:  # 2D
        force = force_2D(coords, dist_mat)
    elif len(coords[0]) == 3:  # 3D
        force = force_3D(coords, dist_mat)

    return force


def update_coords(coords, force, dt):
    """Updates coordinates based on resultant force"""
    return coords + force * dt


def dt_scheduler(dt, itr):
    """Reduces learning rate in iterations"""
    schedule = {200: 1.5, 1000: 2, 2000: 2}
    if itr in schedule.keys():
        dt /= schedule[itr]
    return dt


def run_simulation(init_coords, dist_mat, iters=100, dt=0.1):
    """Runs simulation 'iters' time. Each step calculates forces and updates coordinates"""
    coords = [init_coords, ]
    force = [calculate_force(coords[-1], dist_mat), ]

    for itr_n in range(iters):
        dt = dt_scheduler(dt, itr_n)
        force.append(calculate_force(coords[-1], dist_mat))
        coords.append(update_coords(coords[-1], force[-1], dt))
        if np.max(np.abs(force[-1])) < 1e-4:
            print("Simulation stopped on iteration {}".format(itr_n))
            break

    return np.array(coords), np.array(force)


def plot_performances(coords_list, forces_list):
    """Plot Maximum and Average forces, Average distance error"""
    x = np.arange(len(forces_list)) 

    plt.figure(figsize=(10, 5))
    plt.plot(x, np.max(forces_list, axis=1))
    plt.title('Maximum Force')
    plt.show()

    plt.figure(figsize=(10, 5))
    plt.plot(x, np.average(np.abs(forces_list), axis=1))
    plt.title('Average Force')
    plt.show()

    plt.figure(figsize=(10, 5))
    plt.plot(x[-50:], np.max(forces_list[-50:], axis=1))
    plt.title('Force last 50 iterations')
    plt.show()
  
    diffs_noisy = [np.average(abs(noisy_distances - calc_dist_matrix(c))) for c in coords_list]  # needs a lot of time
    diffs_true = [np.average(abs(sorted_distances - calc_dist_matrix(c))) for c in coords_list]  # needs a lot of time
    plt.figure(figsize=(10, 5))
    plt.plot(x, diffs_noisy, label='Between noisy and simulation')
    plt.plot(x, diffs_true, label='Between True and simulation')
    plt.title('Distance difference')
    plt.legend()
    plt.show()
    
    plt.figure(figsize=(10, 5))
    plt.plot(x[-50:], diffs_noisy[-50:], label='Between noisy and simulation')
    plt.plot(x[-50:], diffs_true[-50:], label='Between True and simulation')
    plt.title('Distance difference')
    plt.legend()
    plt.show()


### Example of device coordinate initialization

Generate initial placement points for **1D** and **2D**. Points are evenly distributed to line (**1D**) and to circle (**2D**). The distance matrix is used to generate actual coordinates in any dimensional space. We have generated a distance matrix for **2D**, but for visualization purposes, we use it to generate starting points for **1D** and **2D**.

In [ ]:
startin_points_1D = generate_starting_points(noisy_distances, n, 1)
startin_points_2D = generate_starting_points(noisy_distances, n, 2)

plt.plot(startin_points_1D.reshape((n)), [0.1]*n, '*', color='blue', label='1D')
plt.plot(*zip(*startin_points_2D), '*', color='green', label='2D')
plt.plot(*zip(*coordinates), 'o', color='orange', label='True Coordinates')
plt.title('Initial placement of points')
plt.legend()
plt.show()

# Main Loop for Simulation <a name="main_loop"></a>

Define the parameters of the problem in the next box. The next cell generates corresponding noisy data runs the simulation. 

After running the simulation 3 errors are generated to estimate performance. Those numbers are MAE between:
* True distances and simulation output
* True distances and noisy distances
* Noisy distances and simulation output 

Most important is the first error between **True distances and simulation output**.  It shows how close our reconstruction is to True Coordinates. We should note that it does not become zero if data has noise. 

After generating final results several graphs are plotted to show some additional metrics performance during simulation. Those plots are:
* Average Absolute Force on points
* Maximum force on a point
* MAE between True distances and simulation output

In [ ]:
dims = 3  # Set dimension for space 1,2 or 3
n = 15 # Set points number
dt = 1 # Set learning rate for mechanical system update. Use range (0.1, 1.5), higher for higher dimension
iters = 1000  # Set maximum number of iterations

In [ ]:
%%time
signal_coordinate = get_coordinates(n, dims)  # Create n device coordinates
distances = calc_dist_matrix(signal_coordinate)  # Calculate distances between points
sorted_distances, sorted_indexes = sort_dist_matrix(distances)  # Sort distance matrix
noisy_distances = add_noise(sorted_distances)  # Add noise
# noisy_distances = sorted_distances  ## Uncomment to use clean data
starting_points = generate_starting_points(noisy_distances, n, dims)  # Initialize starting placement 

coords_list, forces_list = run_simulation(starting_points, noisy_distances, iters, dt)  # Run simulation

# Calculate total absolute difference between noisy distances and true distances before noise
t_n_disp = np.average(abs(sorted_distances-noisy_distances))
# Calculate total absolute difference between true distances and distances calculated using simulation output
t_s_disp = np.average(abs(sorted_distances - calc_dist_matrix(coords_list[-1])))  
# Calculate total absolute difference between noisy distances and distances calculated using simulation output
s_n_disp = np.average(abs(noisy_distances - calc_dist_matrix(coords_list[-1])))  

print('Average error (noise) between true distances and simulation output {:.2f}'.format(t_s_disp))
print('Average error (noise) between true distances and noisy distances {:.2f}'.format(t_n_disp))
print('Average error (noise) between noisy distances and simulation output {:.2f}'.format(s_n_disp))

plot_performances(coords_list, forces_list)

# Animate points movements for 1D and 2D
Animates points movement for 1D and 2D systems. The final point placement result can be translated, mirrored, and rotated from true coordinates. 


In [ ]:
from matplotlib import animation
if dims==1:
    fig, ax = plt.subplots(figsize=(10,3))
    ax.set_xlim(np.min(coords_list)-10, np.max(coords_list)+10)
    ax.set_ylim(0.8, 1.2 )

    x = noisy_distances[0]
    y = np.array([1]*len(x))

    line, = ax.plot(x, y, 'o',  label='Simulation')
    ax.plot(x, y-0.05, 'o', label='Noisy distances')
    ax.plot(list(zip(*signal_coordinate))[0], y-0.1, 'o', label='True Position Before noise')
    ax.plot(starting_points, y+0.1, 'o', label='Initial Placement')

    def animate(i):
        if i < len(coords_list):
            x = coords_list[i] - np.min(coords_list[i])
        else:
            x = coords_list[-1] - np.min(coords_list[-1])

        line.set_xdata(x)  
        return line,


    ani = animation.FuncAnimation(
        fig, animate, interval=100, blit=True, repeat=True, save_count=10)
    plt.legend()
    plt.show()

elif dims==2:
    fig, ax = plt.subplots(figsize=(10,10))
    ax.set_xlim(np.min(coords_list[:,:,0])-50, np.max(coords_list[:,:,0])+50)
    ax.set_ylim(np.min(coords_list[:,:,1])-50, np.max(coords_list[:,:,1])+50)

    x,y = list(zip(*coords_list[0]))

    line, = ax.plot(x, y, 'o',  label='Simulation')
    ax.plot(*list(zip(*coords_list[-1])), 'o', label='Final Placement')
    ax.plot(*list(zip(*signal_coordinate)), '*', label='True Position')

    def animate(i):
        if i < len(coords_list):
            x,y = list(zip(*coords_list[i]))
        else:
            x,y = list(zip(*coords_list[-1]))

        line.set_xdata(x)  
        line.set_ydata(y)  
        return line,

    ani = animation.FuncAnimation(
        fig, animate, interval=10, blit=True, repeat=True, save_count=10)
    plt.legend()
    plt.show()